In [1]:
from utz import *
from nj_crashes.crashes import Crashes

In [2]:
%%time
c = Crashes.load()
c

CPU times: user 22.8 s, sys: 2.51 s, total: 25.3 s
Wall time: 23.3 s


County Name  \
year County Code Municipality Code Department Case Number               
2001 01          01                #2001-17846               ATLANTIC   
                                   01-00029                  ATLANTIC   
                                   01-004615                 ATLANTIC   
                                   01-004880                 ATLANTIC   
                                   01-004912                 ATLANTIC   
...                                                               ...   
2021 21          23                B150-2021-00117A            WARREN   
                                   B150-2021-00118A            WARREN   
                                   B150-2021-00128A            WARREN   
                                   B150-2021-00130A            WARREN   
                                   B150-2021-00132A            WARREN   

                                                          Municipality Name  \
year County Code Municipality Code Department Case Number                     
2001 01          01                #2001-17846                 ABSECON CITY   
                                   01-00029                    ABSECON CITY   
                                   01-004615                   ABSECON CITY   
                                   01-004880                   ABSECON CITY   
                                   01-004912                   ABSECON CITY   
...                                                                     ...   
2021 21          23                B150-2021-00117A               WHITE TWP   
                                   B150-2021-00118A               WHITE TWP   
                                   B150-2021-00128A               WHITE TWP   
                                   B150-2021-00130A               WHITE TWP   
                                   B150-2021-00132A               WHITE TWP   

                                                          Police Department Code  \
year County Code Municipality Code Department Case Number                          
2001 01          01                #2001-17846                                01   
                                   01-00029                                    1   
                                   01-004615                                   1   
                                   01-004880                                   1   
                                   01-004912                                   1   
...                                                                          ...   
2021 21          23                B150-2021-00117A                           02   
                                   B150-2021-00118A                           02   
                                   B150-2021-00128A                           02   
                                   B150-2021-00130A                           02   
                                   B150-2021-00132A                           02   

                                                                 Police Department  \
year County Code Municipality Code Department Case Number                            
2001 01          01                #2001-17846                        ABSECON CITY   
                                   01-00029                                ABSECON   
                                   01-004615                               ABSECON   
                                   01-004880                               ABSECON   
                                   01-004912                               ABSECON   
...                                                                            ...   
2021 21          23                B150-2021-00117A        NEW JERSEY STATE POLICE   
                                   B150-2021-00118A        NEW JERSEY STATE POLICE   
                                   B150-2021-00128A        NEW JERSEY STATE POLICE   
                                   B150-2

Helper that returns:
1. unique sets of values found in `cols`
2. rows from 1. that match all columns but the last

In [3]:
def ambiguous_mappings(df, cols):
    uniqs = (
        df
        .reset_index()
        [cols]
        .drop_duplicates()
        .sort_values(cols)
    )
    hist = uniqs.value_counts(cols[:-1], sort=False)
    conflicts = hist[hist > 1]
    conflicts = uniqs.set_index(cols[:-1]).loc[conflicts.index].reset_index()
    return uniqs, conflicts

## County Code ⟹ County Name

In [4]:
%%time
cn_cols = [ 'County Code', 'County Name' ]
_, county_name_conflicts = ambiguous_mappings(c, cn_cols)
assert len(county_name_conflicts) == 0
c.reset_index()[cn_cols].value_counts(sort=False).rename('# Crashes')

CPU times: user 10.8 s, sys: 10.1 s, total: 20.9 s
Wall time: 25.7 s


County Code  County Name
01           ATLANTIC       192238
02           BERGEN         656350
03           BURLINGTON     264893
04           CAMDEN         324947
05           CAPE MAY        65356
06           CUMBERLAND      98515
07           ESSEX          610767
08           GLOUCESTER     155854
09           HUDSON         424523
10           HUNTERDON       83811
11           MERCER         271333
12           MIDDLESEX      606975
13           MONMOUTH       416092
14           MORRIS         325609
15           OCEAN          340102
16           PASSAIC        390426
17           SALEM           36922
18           SOMERSET       237074
19           SUSSEX          78639
20           UNION          425445
21           WARREN          71432
Name: # Crashes, dtype: int64

## { County Code, Muni Code } ⟹ Muni Name conflicts

In [5]:
%%time
mn_keys = [ 'County Code', 'Municipality Code' ]
mn_val = 'Municipality Name'
mn_cols = mn_keys + [ mn_val ]
_, muni_name_conflicts = ambiguous_mappings(c, mn_cols)
muni_name_conflicts.set_index(mn_cols)

CPU times: user 5.56 s, sys: 4.42 s, total: 9.98 s
Wall time: 12.1 s


Empty DataFrame
Columns: []
Index: [(14, 30, LONG HILL TWP), (14, 30, PASSAIC TWP), (15, 07, DOVER TWP), (15, 07, TOMS RIVER TWP), (16, 16, WEST PATERSON BORO), (16, 16, WOODLAND PARK BORO)]

## { County Code, Muni Code, Year } ⟹ Muni Name

In [6]:
%%time
mny_keys = [ 'County Code', 'Municipality Code', 'year' ]
mny_cols = mny_keys + [ mn_val ]
mny_uniqs, mny_conflicts = ambiguous_mappings(c, mny_cols)
assert len(mny_conflicts) == 0
mny_uniqs.set_index(mn_cols + ['year'])

CPU times: user 5.67 s, sys: 4.92 s, total: 10.6 s
Wall time: 13.2 s


Empty DataFrame
Columns: []
Index: [(01, 01, ABSECON CITY, 2001), (01, 01, ABSECON CITY, 2002), (01, 01, ABSECON CITY, 2003), (01, 01, ABSECON CITY, 2004), (01, 01, ABSECON CITY, 2005), (01, 01, ABSECON CITY, 2006), (01, 01, ABSECON CITY, 2007), (01, 01, ABSECON CITY, 2008), (01, 01, ABSECON CITY, 2009), (01, 01, ABSECON CITY, 2010), (01, 01, ABSECON CITY, 2011), (01, 01, ABSECON CITY, 2012), (01, 01, ABSECON CITY, 2013), (01, 01, ABSECON CITY, 2014), (01, 01, ABSECON CITY, 2015), (01, 01, ABSECON CITY, 2016), (01, 01, ABSECON CITY, 2017), (01, 01, ABSECON CITY, 2018), (01, 01, ABSECON CITY, 2019), (01, 01, ABSECON CITY, 2020), (01, 01, ABSECON CITY, 2021), (01, 02, ATLANTIC CITY, 2001), (01, 02, ATLANTIC CITY, 2002), (01, 02, ATLANTIC CITY, 2003), (01, 02, ATLANTIC CITY, 2004), (01, 02, ATLANTIC CITY, 2005), (01, 02, ATLANTIC CITY, 2006), (01, 02, ATLANTIC CITY, 2007), (01, 02, ATLANTIC CITY, 2008), (01, 02, ATLANTIC CITY, 2009), (01, 02, ATLANTIC CITY, 2010), (01, 02, ATLANTIC CITY, 2011), (01, 02, ATLANTIC CITY, 2012), (01, 02, ATLANTIC CITY, 2013), (01, 02, ATLANTIC CITY, 2014), (01, 02, ATLANTIC CITY, 2015), (01, 02, ATLANTIC CITY, 2016), (01, 02, ATLANTIC CITY, 2017), (01, 02, ATLANTIC CITY, 2018), (01, 02, ATLANTIC CITY, 2019), (01, 02, ATLANTIC CITY, 2020), (01, 02, ATLANTIC CITY, 2021), (01, 03, BRIGANTINE CITY, 2001), (01, 03, BRIGANTINE CITY, 2002), (01, 03, BRIGANTINE CITY, 2003), (01, 03, BRIGANTINE CITY, 2004), (01, 03, BRIGANTINE CITY, 2005), (01, 03, BRIGANTINE CITY, 2006), (01, 03, BRIGANTINE CITY, 2007), (01, 03, BRIGANTINE CITY, 2008), (01, 03, BRIGANTINE CITY, 2009), (01, 03, BRIGANTINE CITY, 2010), (01, 03, BRIGANTINE CITY, 2011), (01, 03, BRIGANTINE CITY, 2012), (01, 03, BRIGANTINE CITY, 2013), (01, 03, BRIGANTINE CITY, 2014), (01, 03, BRIGANTINE CITY, 2015), (01, 03, BRIGANTINE CITY, 2016), (01, 03, BRIGANTINE CITY, 2017), (01, 03, BRIGANTINE CITY, 2018), (01, 03, BRIGANTINE CITY, 2019), (01, 03, BRIGANTINE CITY, 2020), (01, 03, BRIGANTINE CITY, 2021), (01, 04, BUENA BORO, 2001), (01, 04, BUENA BORO, 2002), (01, 04, BUENA BORO, 2003), (01, 04, BUENA BORO, 2004), (01, 04, BUENA BORO, 2005), (01, 04, BUENA BORO, 2006), (01, 04, BUENA BORO, 2007), (01, 04, BUENA BORO, 2008), (01, 04, BUENA BORO, 2009), (01, 04, BUENA BORO, 2010), (01, 04, BUENA BORO, 2011), (01, 04, BUENA BORO, 2012), (01, 04, BUENA BORO, 2013), (01, 04, BUENA BORO, 2014), (01, 04, BUENA BORO, 2015), (01, 04, BUENA BORO, 2016), (01, 04, BUENA BORO, 2017), (01, 04, BUENA BORO, 2018), (01, 04, BUENA BORO, 2019), (01, 04, BUENA BORO, 2020), (01, 04, BUENA BORO, 2021), (01, 05, BUENA VISTA TWP, 2001), (01, 05, BUENA VISTA TWP, 2002), (01, 05, BUENA VISTA TWP, 2003), (01, 05, BUENA VISTA TWP, 2004), (01, 05, BUENA VISTA TWP, 2005), (01, 05, BUENA VISTA TWP, 2006), (01, 05, BUENA VISTA TWP, 2007), (01, 05, BUENA VISTA TWP, 2008), (01, 05, BUENA VISTA TWP, 2009), (01, 05, BUENA VISTA TWP, 2010), (01, 05, BUENA VISTA TWP, 2011), (01, 05, BUENA VISTA TWP, 2012), (01, 05, BUENA VISTA TWP, 2013), (01, 05, BUENA VISTA TWP, 2014), (01, 05, BUENA VISTA TWP, 2015), (01, 05, BUENA VISTA TWP, 2016), ...]

[11747 rows x 0 columns]

## Muni Name transitions

In [7]:
%%time
mnys = (
    mny_uniqs
    .merge(muni_name_conflicts, on=mn_cols)
    .set_index(mny_keys)
    .sort_index()
    [mn_val]
)
mnys

CPU times: user 5.05 ms, sys: 2.64 ms, total: 7.69 ms
Wall time: 9.03 ms


County Code  Municipality Code  year
14           30                 2001           PASSAIC TWP
                                2002           PASSAIC TWP
                                2003           PASSAIC TWP
                                2004           PASSAIC TWP
                                2005           PASSAIC TWP
                                2006           PASSAIC TWP
                                2007           PASSAIC TWP
                                2008           PASSAIC TWP
                                2009           PASSAIC TWP
                                2010           PASSAIC TWP
                                2011           PASSAIC TWP
                                2012           PASSAIC TWP
                                2013         LONG HILL TWP
                                2014         LONG HILL TWP
                                2015         LONG HILL TWP
                                2016         LONG HILL TWP
                   

In [8]:
mnys[mnys != mnys.shift(1)]

County Code  Municipality Code  year
14           30                 2001           PASSAIC TWP
                                2013         LONG HILL TWP
                                2017           PASSAIC TWP
15           07                 2001             DOVER TWP
                                2008        TOMS RIVER TWP
16           16                 2001    WEST PATERSON BORO
                                2008    WOODLAND PARK BORO
Name: Municipality Name, dtype: object

## Verify Muni Codes auto-increment within each County

In [9]:
%%time
codes_hist = c.reset_index()[['County Code', 'Municipality Code']].astype(int).value_counts(sort=False)
codes_hist

CPU times: user 5.72 s, sys: 4.41 s, total: 10.1 s
Wall time: 12.1 s


County Code  Municipality Code
1            1                     7462
             2                    43797
             3                     2958
             4                     2057
             5                     4150
                                  ...  
21           19                   10066
             20                    5204
             21                    2757
             22                    3853
             23                    2918
Length: 568, dtype: int64

In [10]:
def fsck_codes(s):
    l = s.tolist()
    return l == list(range(1, len(l) + 1)), f"{l}"

assert (
    codes_hist
    .reset_index()
    .groupby('County Code')
    ['Municipality Code']
    .apply(fsck_codes)
).all()